In [ ]:
import polars as pl

pl.Config.set_tbl_cols(50)
pl.Config.set_fmt_str_lengths(20)
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_tbl_rows(200)

In [ ]:
df = pl.read_parquet('games.parquet')
print(df.columns)

team1 = 'PIT'
team2 = 'BOS'

df = df.filter(((pl.col('homeTeam') == team1) | (pl.col('awayTeam') == team1)) & ((pl.col('homeTeam') == team2) | (pl.col('awayTeam') == team2)))
df = df.with_columns(
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)
df.group_by('winner').len()

In [ ]:
df = pl.read_parquet('games.parquet')

df = df.with_columns((pl.col('homePim') + pl.col('awayPim')).abs().alias('pim'))
df.sort(pl.col('pim'), descending=True)

In [ ]:
games = pl.read_parquet('games.parquet')
team_stats = pl.read_parquet('team_stats.parquet')
standings = pl.read_parquet('team_standings.parquet').unnest('teamName').rename({'default': 'teamFullName'})

full_team_stats = team_stats.join(standings, on=('seasonId', 'teamFullName'))

# need to aggregate regular season stats and playoff stats here